# NLP (Natural language processing)

In [1]:
import numpy as np
from collections import Counter
import pandas as pd
import nltk
#new line
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
import string
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression

# from sklearn.cross_validation import train_test_split
# New module is
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline  import Pipeline, FeatureUnion, make_pipeline

print("Imported Modules")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alwinsolair/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Imported Modules


In [2]:
stops = set(nltk.corpus.stopwords.words('english'))

In [3]:
#stops

In [4]:
corpus = ["Jeff stole my octopus sandwich.", 
    "'Help!' I sobbed, sandwichlessly.", 
    "'Drop the sandwiches!' said the sandwich police."]

## How do I turn a corpus of documents into a feature matrix?

**Words --> numbers?????**

**Corpus: list of documents**

 [
     "Jeff stole my octopus sandwich.", 
     "'Help!' I sobbed, sandwichlessly.", 
     "'Drop the sandwiches!' said the sandwich police."
 ]

In [5]:
def our_tokenizer(doc, stops=None, stemmer=None):
    doc = word_tokenize(doc.lower())
    tokens = [''.join([char for char in tok if char not in string.punctuation]) for tok in doc]
    tokens = [tok for tok in tokens if tok]
    if stops:
        tokens = [tok for tok in tokens if (tok not in stops)]
    if stemmer:
        tokens = [stemmer.stem(tok) for tok in tokens]
    return tokens

In [6]:
tokenized_docs = [our_tokenizer(doc) for doc in corpus]
tokenized_docs

[['jeff', 'stole', 'my', 'octopus', 'sandwich'],
 ['help', 'i', 'sobbed', 'sandwichlessly'],
 ['drop', 'the', 'sandwiches', 'said', 'the', 'sandwich', 'police']]

**Step 1: lowercase, lose punction, split into tokens**

    [
     ['jeff', 'stole', 'my', 'octopus', 'sandwich'],
     ['help', 'i', 'sobbed', 'sandwichlessly'],
     ['drop', 'the', 'sandwiches', 'said', 'the', 'sandwich', 'police']
    ]

In [7]:
stopwords = set(nltk.corpus.stopwords.words('english'))

In [8]:
'i' in stopwords

True

In [9]:
tokenized_docs = [our_tokenizer(doc, stops=stopwords) for doc in corpus]
tokenized_docs

[['jeff', 'stole', 'octopus', 'sandwich'],
 ['help', 'sobbed', 'sandwichlessly'],
 ['drop', 'sandwiches', 'said', 'sandwich', 'police']]

**Step 2: remove stop words**

    [
     ['jeff', 'stole', 'octopus', 'sandwich'],
     ['help', 'sobbed', 'sandwichlessly'],
     ['drop', 'sandwiches', 'said', 'sandwich', 'police']
    ]

In [10]:
tokenized_docs = [our_tokenizer(doc, stops=stopwords, stemmer=SnowballStemmer('english')) for doc in corpus]
tokenized_docs

[['jeff', 'stole', 'octopus', 'sandwich'],
 ['help', 'sob', 'sandwichless'],
 ['drop', 'sandwich', 'said', 'sandwich', 'polic']]

**Step 3: Stemming/Lemmatization**

    [
     ['jeff', 'stole', 'octopus', 'sandwich'],
     ['help', 'sobbed', 'sandwichlessly'],
     ['drop', u'sandwich', 'said', 'sandwich', 'police']
    ]

**OK now what?**

Vocabulary:

    ['drop', 'help', 'jeff', 'octopus', 'police', 'said', 'sandwich', 'sandwichlessly', 'sobbed', 'stole']


In [11]:
vocab_set = set()

In [12]:
for doc in tokenized_docs:
    vocab_set.update(doc)

In [14]:
vocab = sorted(list(vocab_set))
print(vocab)

['drop', 'help', 'jeff', 'octopus', 'polic', 'said', 'sandwich', 'sandwichless', 'sob', 'stole']


### Section 5.2: Count Vectorizer, TFIDF

Count vectorization

***Vocabulary:***

    ['drop', 'help', 'jeff', 'octopus', 'police', 'said', 'sandwich', 'sandwichlessly', 'sobbed', 'stole']
    ['jeff', 'stole', 'octopus', 'sandwich']
    [0, 0, 1, 1, 0, 0, 1, 0, 0, 1]

    ['help', 'sobbed', 'sandwichlessly']
    [0, 1, 0, 0, 0, 0, 0, 1, 1, 0]

    ['drop', u'sandwich', 'said', 'sandwich', 'police']
    [1, 0, 0, 0, 1, 1, 2, 0, 0, 0]
    
**Term frequency**

$$TF_{word,document} = \frac{\#\_of\_times\_word\_appears\_in\_document}{total\_\#\_of\_words\_in\_document}$$

    ['jeff', 'stole', 'octopus', 'sandwich']
    [0, 0, 1/4, 1/4, 0, 0, 1/4, 0, 0, 1/4]

    ['help', 'sobbed', 'sandwichlessly']
    [0, 1/3, 0, 0, 0, 0, 0, 1/3, 1/3, 0]

    ['drop', u'sandwich', 'said', 'sandwich', 'police']
    [1/5, 0, 0, 0, 1/5, 1/5, 2/5, 0, 0, 0]

### Document frequency

$$ DF_{word} = \frac{\#\_of\_documents\_containing\_word}{total\_\#\_of\_documents} $$
Vocabulary:

    ['drop', 'help', 'jeff', 'octopus', 'police', 'said', 'sandwich', 'sandwichlessly', 'sobbed', 'stole']

**Document frequency for each word:**

    [1/3, 1/3, 1/3, 1/3, 1/3, 1/3, 2/3, 1/3, 1/3, 1/3]

### Inverse document frequency

$$ IDF_{word} = \log\left(\frac{total\_\#\_of\_documents}{\#\_of\_documents\_containing\_word}\right) $$

**Vocabulary:**

    ['drop', 'help', 'jeff', 'octopus', 'police', 'said', 'sandwich', 'sandwichlessly', 'sobbed', 'stole']

**IDF for each word:**

    [1.099, 1.099, 1.099, 1.099, 1.099, 1.099, 0.405, 1.099, 1.099, 1.099]

### TFIDF

**Vocabulary:**

    ['drop', 'help', 'jeff', 'octopus', 'police', 'said', 'sandwich', 'sandwichlessly', 'sobbed', 'stole']

**TF * IDF:**

    ['jeff', 'stole', 'octopus', 'sandwich']
    [0, 0, 0.275, 0.275, 0, 0, 0.101, 0, 0, 0.275]

    ['help', 'sobbed', 'sandwichlessly']
    [0, 0.366, 0, 0, 0, 0, 0, 0.366, 0.366, 0]

    ['drop', u'sandwich', 'said', 'sandwich', 'police']
    [0.22, 0, 0, 0, 0.22, 0.22, 0.162, 0, 0, 0]

Now that we have turned our DOCUMENTS into VECTORS, we can put them into whatever machine learning algorithm we want! 
We can use whatever kind of similarity measure we please!

Wow!

In [15]:
cosine_similarity([[0, 0, 0.275, 0.275, 0, 0, 0.101, 0, 0, 0.275],  [0.22, 0, 0, 0, 0.22, 0.22, 0.162, 0, 0, 0]])

array([[1.        , 0.08115802],
       [0.08115802, 1.        ]])

In [16]:
cosine_similarity([[0, 0.366, 0, 0, 0, 0, 0, 0.366, 0.366, 0],  [0.22, 0, 0, 0, 0.22, 0.22, 0.162, 0, 0, 0]])

array([[1., 0.],
       [0., 1.]])